In [34]:
import pandas as pd
import numpy as np

df = pd.read_csv("2019.csv", encoding='iso-8859-1')
df_new = df.copy()

df_new.drop(columns=df_new.iloc[:,6:].columns, inplace=True)
df_new.drop(columns=["Location", "Latitude", "Longtitude"], inplace=True)

df_new["No. of Cases"] = pd.to_numeric(df_new["No. of Cases"], errors='coerce')

df_new = df_new.groupby([df_new["Closest Location"], df_new.Date]).sum()
station_df = pd.read_csv("station-coordinates.csv")
df_new.head()

No. of Cases
Closest Location Date                    
Admiralty        02/07/2019           2.0
                 03/04/2019          67.0
                 03/05/2019         124.0
                 03/06/2019         343.0
                 04/01/2019           2.0

In [35]:
import requests
import json
from datetime import date
from datetime import datetime, timedelta

date_list = []

row = df_new.index[0]

datetime_object = datetime.strptime(row[1], "%d/%m/%Y")

dt = datetime.combine(datetime_object, datetime.min.time())
dt = dt - timedelta(weeks=2)
timestamp = datetime.timestamp(dt)

station = station_df[station_df["Location"] == row[0]]

lat = float(station["Latitude"])
long = float(station["Longtitude"])
time = int(timestamp)

url = "https://api.darksky.net/forecast/477bf5a88e50f201a476d20e506afa12/{},{},{}".format(lat, long, time)
r = requests.get(url)

data = json.loads(r.text)
print(data['daily'])

data = json.loads(r.text)
data = data['daily']['data'][0]

{'data': [{'time': 1560787200, 'summary': 'Light rain in the morning.', 'icon': 'rain', 'sunriseTime': 1560812460, 'sunsetTime': 1560856380, 'moonPhase': 0.54, 'precipIntensity': 0.0159, 'precipIntensityMax': 0.091, 'precipIntensityMaxTime': 1560815940, 'precipProbability': 0.8, 'precipType': 'rain', 'temperatureHigh': 84.36, 'temperatureHighTime': 1560827100, 'temperatureLow': 78.62, 'temperatureLowTime': 1560880860, 'apparentTemperatureHigh': 97.35, 'apparentTemperatureHighTime': 1560827220, 'apparentTemperatureLow': 83.36, 'apparentTemperatureLowTime': 1560880920, 'dewPoint': 78.4, 'humidity': 0.91, 'pressure': 1010.3, 'windSpeed': 4.92, 'windGust': 10.35, 'windGustTime': 1560828240, 'windBearing': 149, 'cloudCover': 0.95, 'uvIndex': 5, 'uvIndexTime': 1560834420, 'visibility': 9.327, 'ozone': 260.6, 'temperatureMin': 76.48, 'temperatureMinTime': 1560814980, 'temperatureMax': 84.36, 'temperatureMaxTime': 1560827100, 'apparentTemperatureMin': 78.23, 'apparentTemperatureMinTime': 15608

In [37]:
data['precipIntensity']
data['precipIntensityMax']
data['precipIntensityMaxTime']
data['temperatureHigh']
data['temperatureHighTime']
data['temperatureLow']
data['temperatureLowTime']
data['humidity']

0.91

In [38]:
precipIntensity = []
precipIntensityMax = []
precipIntensityMaxTime = []
temperatureHigh = []
temperatureHighTime = []
temperatureLow = []
temperatureLowTime = []
humidity = []

precipIntensity.append(data['precipIntensity'])
precipIntensityMax.append(data['precipIntensityMax'])
precipIntensityMaxTime.append(data['precipIntensityMaxTime'])
temperatureHigh.append(data['temperatureHigh'])
temperatureHighTime.append(data['temperatureHighTime'])
temperatureLow.append(data['temperatureLow'])
temperatureLowTime.append(data['temperatureLowTime'])
humidity.append(data['humidity'])

In [39]:
from datetime import date
from datetime import datetime, timedelta
import requests
import json

date_list = []
precipIntensity = []
precipIntensityMax = []
precipIntensityMaxTime = []
temperatureHigh = []
temperatureHighTime = []
temperatureLow = []
temperatureLowTime = []
humidity = []

for row in df_new.iterrows():
    
    if([row[0][0], row[0][1]] not in date_list):
        datetime_object = datetime.strptime(row[0][1], "%d/%m/%Y")

        dt = datetime.combine(datetime_object, datetime.min.time())
        dt = dt - timedelta(weeks=2)
        timestamp = datetime.timestamp(dt)

        station = station_df[station_df["Location"] == row[0][0]]

        lat = float(station["Latitude"])
        long = float(station["Longtitude"])
        time = int(timestamp)

        url = "https://api.darksky.net/forecast/477bf5a88e50f201a476d20e506afa12/{},{},{}".format(lat, long, time)
        r = requests.get(url)

        data = json.loads(r.text)
        data = data['daily']['data'][0]

        precipIntensity.append(data['precipIntensity'])
        precipIntensityMax.append(data['precipIntensityMax'])
        precipIntensityMaxTime.append(data['precipIntensityMaxTime'])
        temperatureHigh.append(data['temperatureHigh'])
        temperatureHighTime.append(data['temperatureHighTime'])
        temperatureLow.append(data['temperatureLow'])
        temperatureLowTime.append(data['temperatureLowTime'])
        humidity.append(data['humidity'])

        date_list.append([row[0][0], row[0][1]])
    

In [194]:
precip_list = []
for date in date_list:
    for index in df_new.index:
        if date == list(index):
            count = date_list.index(date)
            precip_list.append(precipIntensity[count])

In [40]:
df_new['precipIntensity'] = precipIntensity
df_new['precipIntensityMax'] = precipIntensityMax
df_new['precipIntensityMaxTime'] = precipIntensityMaxTime
df_new['temperatureHigh'] = temperatureHigh
df_new['temperatureHighTime'] = temperatureHighTime
df_new['temperatureLow'] = temperatureLow
df_new['temperatureLowTime'] = temperatureLowTime
df_new['humidity'] = humidity

In [41]:
df_new.to_csv('2019_done.csv', index=True)